In [1]:
import os
import pandas as pd
import numpy as np
from lib.dfInfo import get_df_info, show_df_info_bar
import plotly.io as pio
import plotly.express as px
from pyvis.network import Network
import networkx as nx

from base64 import b64encode
from io import BytesIO

from IPython.display import Markdown as md, display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

import chart_studio.plotly as tls
import plotly.graph_objects as go
from scipy.stats import skew, kurtosis, chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from scipy import stats

from Levenshtein import ratio

import itertools

from lib import pipeUtils
pio.renderers.default = "iframe"

working_dir = "working"
if not os.path.exists(working_dir):
    os.mkdir(working_dir)


df = pd.read_csv("assets/2016_Building_Energy_Benchmarking.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [3]:
df.shape

(3376, 46)

3376 observations, on remarque une feature non remplie à 100%: Comments ainsi que 2 features très peu remplies: YearsENERGYSTARCertified et Outlier

In [4]:
df_info = get_df_info(df)
show_df_info_bar(df_info=df_info)

# Filtrage

In [5]:
df["BuildingType"].unique()

array(['NonResidential', 'Nonresidential COS', 'Multifamily MR (5-9)',
       'SPS-District K-12', 'Campus', 'Multifamily LR (1-4)',
       'Multifamily HR (10+)', 'Nonresidential WA'], dtype=object)

In [6]:
df["PrimaryPropertyType"].unique()

array(['Hotel', 'Other', 'Mid-Rise Multifamily', 'Mixed Use Property',
       'K-12 School', 'University', 'Small- and Mid-Sized Office',
       'Self-Storage Facility', 'Warehouse', 'Large Office',
       'Senior Care Community', 'Medical Office', 'Retail Store',
       'Hospital', 'Residence Hall', 'Distribution Center',
       'Worship Facility', 'Low-Rise Multifamily',
       'Supermarket / Grocery Store', 'Laboratory',
       'Refrigerated Warehouse', 'Restaurant', 'High-Rise Multifamily',
       'Office'], dtype=object)

In [7]:
df_residences = df.loc[~((df["BuildingType"]!="Multifamily MR (5-9)") & (df["BuildingType"]!="Multifamily LR (1-4)") & (df["BuildingType"]!="Multifamily HR (10+)") & (df["PrimaryPropertyType"]!="Low-Rise Multifamily"))].reset_index(drop=True)

In [8]:
df = df.loc[(df["BuildingType"]!="Multifamily MR (5-9)") & (df["BuildingType"]!="Multifamily LR (1-4)") & (df["BuildingType"]!="Multifamily HR (10+)") & (df["PrimaryPropertyType"]!="Low-Rise Multifamily")].reset_index(drop=True)

In [9]:
df_info = get_df_info(df)
show_df_info_bar(df_info=df_info)

In [19]:
fig = go.Figure(
    go.Scattermapbox(
        lat = df['Latitude'],
        lon = df['Longitude'],
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            color="fuchsia",
            opacity=0.5
        )
    )
)

fig.add_trace(
    go.Scattermapbox(
        mode = "markers",
        lon = df_residences['Longitude'],
        lat = df_residences['Latitude'],
        marker = go.scattermapbox.Marker(
            color= 'blue', 
            opacity= 0.5
        )
    )
)

fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0,"t":0,"l":0,"b":0},
    height= 800,
    mapbox = {
        "zoom":12,
        "center": go.layout.mapbox.Center(
            lat=df["Latitude"].mean(),
            lon=df["Longitude"].mean()
        ),

    }
)


SyntaxError: ':' expected after dictionary key (3447231246.py, line 31)

In [11]:
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude",
                        color_discrete_sequence=["fuchsia"], zoom=12, height=900)
fig.update_layout(")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

SyntaxError: unterminated string literal (detected at line 3) (211008911.py, line 3)

# Nettoyage et imputation

In [ ]:
df = pipeUtils.clean_and_impute(df, inplace=True)


In [ ]:
df.info()

# Analyse univariée

In [ ]:
num_cols = []
cat_cols = []
for c in df.columns:
    if df.dtypes[c]=="float64" or df.dtypes[c]=="int64":
        num_cols.append(c)
    elif df.dtypes[c]=="object":
        cat_cols.append(c)
        
def add_split_screen(title, html, figsdata=[], figshtml=[], iwidth=None):
    
    iwidth = ' width={0} '.format(iwidth) if iwidth is not None else ''
    datatable = f"""<table><tr><td colspan="{1 + len(figsdata) + len(figshtml)}" align="center"><h3>{title}</h3></td></tr><tr><td>{html}</td>"""
    for figdata in figsdata:
        datatable += f"""<td><img src="data:image/png;base64,{b64encode(figdata.getvalue()).decode()}"/></td>"""
    for fightml in figshtml:
        datatable += f"""<td>{fightml}</td>"""
    datatable += """</tr></table>"""

    display(HTML(datatable)) 


df0 = df.copy()
for c in num_cols:
    continue
    if c=="OSEBuildingID":
        continue
    
    df0[c] = df0[c].astype(float)
    
    d = df0.loc[:,c].describe().to_dict()
    d_html = "<table>"
    for t in d:
        d_html += f"""<tr><td>{t}</td><td align="right">{d[t]}</td></tr>"""
    d_html += f"""<tr><td>skewness</td><td align="right">{skew(df0.loc[~df0[c].isna(), c])}</td></tr>"""
    k = kurtosis(df0.loc[~df0[c].isna(), c])
    d_html += f"""<tr><td>kurtosis</td><td align="right">{k}</td></tr>"""
    d_html += "</table>"
    
    fig = px.box(df0, x=c)
    fig.update_layout(
        width=500,
        height=350
    )
    fig1html = fig.to_html()

    fig = px.histogram(df0, x=c, log_y=False)
    fig.update_layout(
        width=500,
        height=350
    )
    fig2html = fig.to_html()

    if k>8:
        fig = px.histogram(df0, x=c, log_y=True)
        fig.update_layout(
            width=500,
            height=350
        )
        fig3html = fig.to_html()
        add_split_screen(c, d_html, figshtml=[fig1html, fig2html, fig3html])
    else:
        add_split_screen(c, d_html, figshtml=[fig1html, fig2html])


In [ ]:
for c in cat_cols:
    if len(df[c].unique())>30:
        continue
    f = plt.figure(figsize=(25,3))
    #plt.subplot(1,1,1)
    g = sns.countplot(x=df.sort_values(c)[c])
    plt.xticks(rotation=90)
    f.suptitle(c)
    plt.show()

In [ ]:
target_cols = ["SiteEUIWN(kBtu/sf)", "GHGEmissionsIntensity"]

In [ ]:
num_cols = []
for c in df.columns:
    if df.dtypes[c]=="float64" or df.dtypes[c]=="int64":
        num_cols.append(c)

cols_to_logify = []
for c in num_cols:
    if c=="OSEBuildingID" or c in target_cols:
        continue
    
    df0[c] = df0[c].astype(float)
    k = kurtosis(df0.loc[~df0[c].isna(), c])
    if k>8 and c.endswith("_ratio")==False:
        cols_to_logify.append(c)
print(f"""Colonnes à transformer en log: {cols_to_logify}""")

In [ ]:
for col in cols_to_logify:
    df["log_"+col] = np.log(df[col] + 1)
df.drop(columns=cols_to_logify, inplace=True)

In [ ]:
#df.drop(columns=["ENERGYSTARScore"], inplace=True)

In [ ]:
num_cols = []
for c in df.columns:
    if df.dtypes[c]=="float64" or df.dtypes[c]=="int64":
        num_cols.append(c)


In [ ]:
n_cols = df.select_dtypes(['int32', 'float64']).columns
df_pp = df.loc[:, n_cols].dropna()

In [ ]:
df_pp.info()

In [ ]:
# Nordine : to remove ?
#df["SiteEUIWN(kBtu/sf)"] = np.log(df["SiteEUIWN(kBtu/sf)"]+1)
#df["GHGEmissionsIntensity"] = np.log(df["GHGEmissionsIntensity"]+1)

In [ ]:
fig = px.imshow(df_pp.corr(numeric_only =True))
fig.update_layout(
    autosize=False,
    width=1000,
    height=1000
)
fig.show()

In [ ]:
n_cols = df.select_dtypes(['int32', 'float64']).columns
df_pp = df.loc[:, n_cols].dropna()

g = sns.pairplot(df_pp)
for ax in g.axes.flatten():
    ax.tick_params(rotation = 60)
plt.show()

# Analyse bivariée

## ANOVA

Nous allons réaliser une analyse ANOVA entre les features de consommation energétiques, les features d' émissions et les features catégorielles


In [ ]:
def eta_squared(x,y):
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x==classe]
        classes.append({'ni': len(yi_classe),
                        'moyenne_classe': yi_classe.mean()})
    SCT = sum([(yj-moyenne_y)**2 for yj in y])
    SCE = sum([c['ni']*(c['moyenne_classe']-moyenne_y)**2 for c in classes])
    return SCE/SCT
    

In [ ]:
cat_cols

In [ ]:
df.columns

In [ ]:
df["DecadeBuilt"] = df["DecadeBuilt"].astype(str)

num_cols = []
cat_cols = []
for c in df.columns:
    if df.dtypes[c]=="float64" or df.dtypes[c]=="int64":
        num_cols.append(c)
    elif df.dtypes[c]=="object":
        cat_cols.append(c)
        

In [ ]:
box_cols_num = dict()

df_eta = pd.DataFrame()

for nrj_col in target_cols:
    out = f"### {nrj_col}\n| Feature | ETA² | Corrélation avec {nrj_col}|\n"
    out += "|-----------|-----------|-----------|\n"

    etas = {}

    for cat_col in cat_cols:
        etas[cat_col] = eta_squared(df[cat_col], df[nrj_col])
    etas = dict(sorted(etas.items(), key=lambda item: item[1], reverse=True))
    eta2 = {}
    eta2["numeric_col"] = nrj_col
    for c in etas:
        effect_size = "Négligeable"
        eta2[c] = 0
        if etas[c] >= 0.14:
            effect_size = "Grande"
            eta2[c] = 3
            if c not in box_cols_num:
                box_cols_num[c] = []
            box_cols_num[c].append(nrj_col)
        elif etas[c] >= 0.06:
            effect_size = "Moyenne"
            eta2[c] = 2
        elif etas[c] >= 0.01:
            effect_size = "Petite"
            eta2[c] = 1
        out += f"""| {c} | {etas[c]} | {effect_size} |\n"""
    df_eta2 = pd.DataFrame([eta2])
    df_eta = pd.concat([df_eta, df_eta2], ignore_index=True)

    display(md(out))
    

On voit que les distributions de 
- SiteEUIWN(kBtu/sf), 
- GHGEmissionsIntensity 

sont fortement corrélées avec les distributions de **LargestPropertyUseType** et **PrimaryPropertyType**.

GHGEmissionsIntensity est aussi corrélé avec SecondLargestPropertyUseType

In [ ]:
df_eta

In [ ]:

fig = px.imshow(df_eta.iloc[:,range(1, len(df_eta.columns))], y=df_eta["numeric_col"])
fig.show()

log_SiteEUIWN(kBtu/sf) est corrélé à:
- LargestPropertyUseType et PrimaryPropertyType de manière forte
- SecondLargestPropertyUseType de manière moyenne

log_GHGEmissionsIntensity est corrélé à:
- LargestPropertyUseType et PrimaryPropertyType de manière forte

## Tests d'indépendance avec les features catégorielles

In [ ]:
num_cols = []
cat_cols = []
for c in df.columns:
    if df.dtypes[c]=="float64" or df.dtypes[c]=="int64":
        num_cols.append(c)
    elif df.dtypes[c]=="object":
        cat_cols.append(c)
        

In [ ]:
out = ""
p_limit = 0.03
df_indep = pd.DataFrame()
for ref_col in target_cols:
    dict_line = {"ref_col":ref_col}
    y = pd.cut(df[ref_col], 20)
    out += f"""### **Test d'indépendance de {ref_col}**\n"""
    out += "| Feature comparée | p | chi2 | Conclusion |\n|-----|-----|-----|-----|\n"
    for num_col in num_cols:
        if num_col in target_cols:
            continue
        if num_col in ["OSEBuildingID"]:
            continue
        if num_col==ref_col:
            continue
        if len(df[num_col].unique())>20:
            x = pd.cut(df[num_col], 20).astype('category')
        else:
            x = df[num_col].astype('category')
        ct = pd.crosstab(x, y)
        stat_chi2, p, dof, expected_table = stats.chi2_contingency(ct)
        out += f"| {num_col} | {p} | {stat_chi2} |"
        if p<p_limit:
            out += f"{ref_col} et {num_col} ne sont pas indépendantes|\n"
            dict_line[num_col] = "1"
        else:
            out += f"{ref_col} et {num_col} sont indépendantes|\n"
            dict_line[num_col] = "0"
    out += "\n\n"
    df_indep2 = pd.DataFrame([dict_line])
    df_indep = pd.concat([df_indep, df_indep2], ignore_index=True)
display(md(out))


In [ ]:
df_indep

In [ ]:
for col in df_indep.columns:
    df_indep[col] = df_indep[col].astype(str)
fig = px.imshow(df_indep.iloc[:,range(1, len(df_indep.columns))], y=df_indep["ref_col"], title="toto")
fig.show()

On voit que les données liées au parking sont indépendantes des features cibles.

log_SiteEUIWN(kBtu/sf) est en relation avec:
- DecadeBuilt (ramené en entier)
- gas_ratio
- electricity_ratio
- log_NumberofFloors
- log_PropertyGFATotal
- log_PropertyGFABuilding(s)
- log_LargestPropertyUseTypeGFA
- log_SecondLargestPropertyUseTypeGFA


GHGEmissionsIntensity est en relation avec:
- DecadeBuilt (transformé en entier)
- gas_ratio
- electricity_ratio
- steam_ratio
- log_NumberofBuildings
- log_PropertyGFATotal
- log_PropertyGFABuilding(s)
- log_LargestPropertyUseTypeGFA
- log_SecondLargestPropertyUseTypeGFA
- log_ThirdLargestPropertyUseTypeGFA


# Analyse PCA

Pour la PCA, nous allons prendre en compte la décennie de construction des immeubles

In [ ]:
df.info()

In [ ]:
df["DecadeBuilt"] = df["DecadeBuilt"].astype(int)

In [ ]:
num_cols = []
cat_cols = []
for c in df.columns:
    if df.dtypes[c]=="float64" or df.dtypes[c]=="int64":
        if c!="ENERGYSTARScore" and c!="log_TotalGHGEmissions" and c!="log_SiteEnergyUse(kBtu)":
            num_cols.append(c)
    elif df.dtypes[c]=="object":
        cat_cols.append(c)


In [ ]:
X = df.loc[:, num_cols].reset_index(drop=True).values

In [ ]:
num_cols

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
n_components = len(num_cols)
pca = PCA(n_components=n_components)
pca.fit(X_scaled)
pca.explained_variance_ratio_

In [ ]:
pca.components_

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=list(range(1, n_components+1)),
        y=pca.explained_variance_ratio_ * 100,
        hoverinfo='x+y',
        name="Variance expliquée" 
    ),
)
fig.add_trace(
    go.Scatter(
        x=list(range(1, n_components+1)), 
        y=np.cumsum(pca.explained_variance_ratio_*100),
        mode='lines',
        name='Variance expliquée cumulée'
    )
)

fig.update_layout(
    title=f"""Ebouli des valeurs propres avec {n_components} composantes""",
    title_x=0.5,
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        yanchor="middle",
        y=0.5,
        xanchor="right",
        x=0.99
    )
)
fig.show()

In [ ]:

def correlation_graph2(pca, x_y, features): 
    colors_cycle = itertools.cycle(px.colors.qualitative.Dark24)

    x,y=x_y

    fig = go.Figure()
    fig.add_shape(
        type="circle",
        xref="x", yref="y",
        x0=-1, y0=-1, x1=1, y1=1,
        line_color="Black",
    )
    
    for i in range(0, pca.components_.shape[1]):
        color = next(colors_cycle)
        fig.add_annotation(
            x=pca.components_[x, i],
            y=pca.components_[y, i],
            xref="x", yref="y",
            showarrow=True,
            axref = "x", ayref='y',
            ax=0,
            ay=0,
            arrowhead=4,
            arrowwidth=2,
            arrowcolor=color,
            font=dict(
                color=color,
                size=12
            ),
        )
        fig.add_trace(
            go.Scatter(
                x=[pca.components_[x, i]],
                y=[pca.components_[y, i]],
                text=[features[i]],
                mode='text',
                marker=dict(
                    color = color
                ),
                name=features[i]
            ), 
        )
        
    fig.update_layout(width=1500, height=1000,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99,
        )
    )
    fig.update_xaxes(range=[-1.1, 1.1], title=dict(text = f"F{x+1}"))
    fig.update_yaxes(range=[-1.1, 1.1], title=dict(text = f"F{y+1}"))
    
    fig.for_each_trace(lambda t: t.update(textfont_color=t.marker.color))
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
    )
    fig.show()

In [ ]:
correlation_graph2(pca, (0,1), num_cols)

On voit que F2 est fortement corrélé aux features cibles ainsi qu'aux ratio d'energies utilisées.

GHGEmissionIntensity est inversement corrélé à electricity_ratio alors qu'il est corrélé dans le même sens à gas_ratio

In [ ]:
correlation_graph2(pca, (2,3), num_cols)

Selon F3 et F4, les features cibles évoluent avec PropertyGFATotal, LargestPropertyUseTypeGFA et NumberofBuildings

In [ ]:
for j in range(0, len(target_cols)):
    df_proj = pd.DataFrame(pca.transform(X_scaled), columns=[f"F{i}" for i in range(1, n_components+1)] )
    df_proj[target_cols[j]] = df[target_cols[j]]
    
    sns.scatterplot(data=df_proj, x="F1", y="F2", hue=target_cols[j])
    plt.show()
    
    sns.scatterplot(data=df_proj, x="F3", y="F4", hue=target_cols[j])
    plt.show()


# Conclusion

L'analyse exploratoire permet de montrer que:

**log_SiteEUIWN(kBtu/sf)** est corrélé à:
- LargestPropertyUseType et PrimaryPropertyType de manière forte
- SecondLargestPropertyUseType de manière moyenne
- DecadeBuilt (ramené en entier)
- gas_ratio
- electricity_ratio
- log_NumberofFloors
- log_PropertyGFATotal
- log_PropertyGFABuilding(s)
- log_LargestPropertyUseTypeGFA
- log_SecondLargestPropertyUseTypeGFA

**log_GHGEmissionsIntensity** est corrélé à:
- LargestPropertyUseType et PrimaryPropertyType de manière forte
- DecadeBuilt (transformé en entier)
- gas_ratio
- electricity_ratio
- steam_ratio
- log_NumberofBuildings
- log_PropertyGFATotal
- log_PropertyGFABuilding(s)
- log_LargestPropertyUseTypeGFA
- log_SecondLargestPropertyUseTypeGFA
- log_ThirdLargestPropertyUseTypeGFA


L'analyse PCA nous montre que les features cibles évoluent fortement avec F2, ce qui veut dire que les features cibles:
- grandissent fortement avec:
    - gas_ratio
- baissent fortement avec:
    - electricity_ratio


In [ ]:
df.to_parquet(f"{working_dir}/explored.parquet")